In [2]:
import pandas as pd
import numpy as np
import re
from gensim.models import word2vec, Word2Vec
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
from numpy import argmax
%matplotlib inline

from ckonlpy.tag import Twitter #konlpy twitter는 add_dictionary가 없어서, ckonlpy(customized konlpy) 사용.
twitter = Twitter()

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## 입력데이터

In [156]:
df = pd.read_csv('connector_1.csv', sep='\t', encoding='UTF-8')

In [157]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [158]:
df.shape

(1601, 17)

## 텍스트 전처리

In [159]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df['result_pred'] = df['검토결과'].astype(str).copy()
# df['target_pred'] = df['대상 선정'].astype(str).copy()
# df['charge_pred'] = df['책임구'].astype(str).copy()
# df['division_pred'] = df['구분'].astype(str).copy()

### (1) 오타, 동의어 처리(replace)

replaceword = pd.read_csv('replaceword_final.csv', sep='\t', encoding='UTF-8')

replaceword.head(10)

replaceword.shape

df['clm_prtl_sbc'][488]

Replaceword_list =  replaceword['Replaceword'].tolist()

Missword_list = replaceword['Missword'].tolist()

print(len(Replaceword_list))
print(len(Missword_list))

for i in range(len(Replaceword_list)):
    df['clm_prtl_sbc'] = df['clm_prtl_sbc'].replace(Missword_list[i], Replaceword_list[i], regex=True)
#regex=True 설정을 주면 조사가 붙어있던 말던 단어를 다 바꿔준다.

df['clm_prtl_sbc'].head()

df['clm_prtl_sbc'][488] # '간극'이 '간격'으로 바뀌었는지 확인.(8529 데이터 기준)

df['clm_prtl_sbc'][670]  # '가니시'가 '가니쉬'로 바뀌었는지 확인.(8529 데이터 기준)

### (2) 사용자 단어 정의(User dictionary)

udw = pd.read_csv('Userdict_final.csv', sep='\t', encoding='UTF-8') #Userdictionary에 등록할 Word 불러오기.

udw.head()

udw_list =  udw['Userdict'].tolist() # 단어들만 list로 추출.

for word in udw_list: 
    twitter.add_dictionary(word, 'Noun') # for문을 통해 dictionary에 단어들을 Noun으로 추가.

twitter.pos(df['clm_prtl_sbc'][1]) # '리어', '미장착'이 Noun으로 잘 정의되었는지 확인.

### (3) 명사 추출(Twitter)

def nouns(text):
    text = str(text)
    text = twitter.nouns(text)
    return text

df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(nouns)

df.loc[1, 'clm_prtl_sbc']  #'리어', '미장착' 붙어서 출력되었는지 확인.(8529 데이터 기준)

### (4) 특수문자, 띄어쓰기 전처리

def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text) 
    text = re.sub(' +', ' ', text)
    return text

# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['(LH)','(RH)','(사진첨부)','(우)','(좌)','[C]','A/S','C','C-10000','C-10000','F/LH','F/RH','FR','FRT',
             'Frt','frt','L/H','L/H리어','LH','LH/RH','LH쪽','R/H','R/H리어','R/LH','R/RH','RH','RH쪽','Rr','RR','rr',
             '결과','고객','고객에게','공임','광주','그랜저','그랜져','낫','낮','넥쏘','니로','당','당차량','당차량은','대리점',
             '동영상참조','레이','렌트','리무진','리어LH','리어RH','모닝','모하비','밤','벨로스터','사진','사진참조','사진첨부',
             '상기','상기차량은','새벽','새차','서비스','소하','스타렉스','스팅어','스포티지','신차','신차','심야','싼타페',
             '써비스','쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','아침','야간','언덕','엑센트','열','울산','원인',
             '임시 넘버','임시 번호','입고','자동차','저머검내용','점검/원인','점검결과','점검내용','점검및원인','점검및조치내용',
             '점검점검','점검점검내용','조치결과','조치내용','주간','주재원','주제원','주차장','증상','지하','진입','차량',
             '차량은','철도','첨부','첨부파일','청구','출고','카 마스터','카니발','카렌스','코나','클레임','투싼',
             '팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상','현 상','현산','현상','현상및점검',
             '현상및조치','화성'
]
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)

%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(remove_stopwords)

df['clm_prtl_sbc'].head()

474
474
Wall time: 68 ms
Wall time: 117 ms


0     인도 후 오디오 작동 시 주파수 및 이음 발 생으로 불량 제기 점검 내용 및 점검...
1     주행 중 엔진 경고등 점등 점검 내용 및 스캔 점검 효율 저하 발생 임시 코드 주...
2                   조수석 사이드 어백 경고등 점등 커넥터 터 체결 불량 재체결 
3                               작동 불량 커넥터 터 조립 불량 재조립 
4                             어라운드뷰 장착 운전석 미러 카메라 미장착 
Name: clm_prtl_sbc, dtype: object

## X, y 정의

In [160]:
from sklearn.preprocessing import LabelEncoder
y_encod = df['검토결과']
classle = LabelEncoder()
y_encod = classle.fit_transform(df['검토결과'].values) # y값 인코딩.
print(np.unique(y_encod))

[0 1 2 3 4]


In [161]:
# ro column을 list형태로 ro_text 변수에 할당.
ro_text = list(df['clm_prtl_sbc'])
sentiments = list(y_encod) # y값.

In [162]:
ro_text[0]

' 인도 후 오디오 작동 시 주파수 및 이음 발 생으로 불량 제기 점검 내용 및 점검 주파수 수신 라디오 주파수 것 확인 나 현대 웰슨 주재 원 필러 탈거 후 오디오 및 케이블 확인 정상 실내 헤드라이닝 탈거 후 콤비네이션 안테나 확인 배선 미체결 있음 조치 내용 콤비네이션 안테나 배션 체결 후 라디오 확인 정상 작동 '

In [163]:
sentiments[:10]

[3, 2, 1, 3, 2, 1, 1, 4, 1, 0]

In [164]:
sentences = []
for text in ro_text:
    sentences.append(text.split())

In [165]:
sentences[:3]

[['인도',
  '후',
  '오디오',
  '작동',
  '시',
  '주파수',
  '및',
  '이음',
  '발',
  '생으로',
  '불량',
  '제기',
  '점검',
  '내용',
  '및',
  '점검',
  '주파수',
  '수신',
  '라디오',
  '주파수',
  '것',
  '확인',
  '나',
  '현대',
  '웰슨',
  '주재',
  '원',
  '필러',
  '탈거',
  '후',
  '오디오',
  '및',
  '케이블',
  '확인',
  '정상',
  '실내',
  '헤드라이닝',
  '탈거',
  '후',
  '콤비네이션',
  '안테나',
  '확인',
  '배선',
  '미체결',
  '있음',
  '조치',
  '내용',
  '콤비네이션',
  '안테나',
  '배션',
  '체결',
  '후',
  '라디오',
  '확인',
  '정상',
  '작동'],
 ['주행',
  '중',
  '엔진',
  '경고등',
  '점등',
  '점검',
  '내용',
  '및',
  '스캔',
  '점검',
  '효율',
  '저하',
  '발생',
  '임시',
  '코드',
  '주행',
  '점검',
  '경고등',
  '점등',
  '조치',
  '내용',
  '및',
  '차압',
  '센서',
  '교환',
  '후',
  '시운전',
  '정상',
  '스로틀',
  '플랩',
  '쪽',
  '호스',
  '미장착'],
 ['조수석', '사이드', '어백', '경고등', '점등', '커넥터', '터', '체결', '불량', '재체결']]

## Word2Vec

In [166]:
# Word2Vec의 파라미터 값들 정의.
num_features = 300    
min_word_count = 40   
num_workers = 4       
context = 10          
downsampling = 1e-3 

In [167]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [168]:
# word2vec으로 단어 학습.
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2019-06-03 15:35:18,969 : INFO : collecting all words and their counts
2019-06-03 15:35:18,970 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-03 15:35:18,980 : INFO : collected 2625 word types from a corpus of 49922 raw words and 1601 sentences
2019-06-03 15:35:18,980 : INFO : Loading a fresh vocabulary
2019-06-03 15:35:18,982 : INFO : effective_min_count=40 retains 185 unique words (7% of original 2625, drops 2440)
2019-06-03 15:35:18,983 : INFO : effective_min_count=40 leaves 38548 word corpus (77% of original 49922, drops 11374)
2019-06-03 15:35:18,984 : INFO : deleting the raw counts dictionary of 2625 items
2019-06-03 15:35:18,984 : INFO : sample=0.001 downsamples 84 most-common words
2019-06-03 15:35:18,985 : INFO : downsampling leaves estimated 18571 word corpus (48.2% of prior 38548)
2019-06-03 15:35:18,986 : INFO : estimated required memory for 185 words and 300 dimensions: 536500 bytes
2019-06-03 15:35:18,986 : INFO : resetting layer weight

In [169]:
# Word2Vec의 feature vector를 얻는 함수.
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [170]:
def get_dataset(ro_text, model, num_features):
    dataset = list()

    for s in ro_text:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [171]:
test_data_vecs = get_dataset(sentences, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [172]:
test_data_vecs

array([[ 0.14391038,  0.06319516,  0.04195809, ...,  0.05927512,
        -0.00468138, -0.0206914 ],
       [ 0.13463518,  0.05151737, -0.00287756, ...,  0.0343424 ,
        -0.01252692, -0.04799657],
       [ 0.14731218,  0.06501707,  0.05999668, ...,  0.05769866,
        -0.01545575, -0.01111347],
       ...,
       [ 0.15549299,  0.08519303,  0.11734617, ...,  0.08354083,
        -0.01104988,  0.02740357],
       [ 0.14190996,  0.05485194,  0.00082672, ...,  0.03781898,
        -0.01224128, -0.04784644],
       [ 0.14243391,  0.05782555,  0.01412835, ...,  0.04592244,
        -0.01036618, -0.04032815]], dtype=float32)

### Train, Test data Split

In [173]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [174]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1280, 300)
(321, 300)
(1280,)
(321,)


In [175]:
np.where(np.isnan(X_train))

(array([], dtype=int64), array([], dtype=int64))

In [176]:
np.where(np.isnan(X_test))

(array([], dtype=int64), array([], dtype=int64))

In [177]:
np.where(np.isnan(y_train))

(array([], dtype=int64),)

In [178]:
np.where(np.isnan(y_test))

(array([], dtype=int64),)

In [179]:
np.where(np.isnan(X_train))

(array([], dtype=int64), array([], dtype=int64))

## RandomForestClassifier로 학습

In [180]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=42)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [181]:
%time forest = forest.fit(X_train, y_train)

Wall time: 345 ms


## 모델평가

In [182]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 4.35 s


array([0.8828125 , 0.8828125 , 0.82421875, 0.87109375, 0.8125    ])

In [183]:
round(np.mean(score)*100,2)

85.47

## 예측 결과

In [184]:
from sklearn.metrics import accuracy_score
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(X_test)
y_pred[:3]

array([0, 1, 0])

In [185]:
print(accuracy_score(y_test, y_pred))

0.8878504672897196
